In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("train/train.csv")

In [3]:
df.columns

Index(['ID', 'y', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8',
       ...
       'X375', 'X376', 'X377', 'X378', 'X379', 'X380', 'X382', 'X383', 'X384',
       'X385'],
      dtype='object', length=378)

In [4]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(df_train, )

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [5]:
df_test = pd.read_csv("test/test.csv")

In [6]:
len(df)

4209

In [7]:
len(df_test)

4209

In [8]:
df_test.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


In [9]:
len(df_test.columns)

377

In [10]:
len(df.columns)

378

In [11]:
len(df.columns)

378

In [12]:
col_drop = []

for col in df.columns:
    if (len(df[col].unique())) == 1:
        col_drop.append(col)

In [13]:
df.drop(col_drop, axis=1, inplace=True)
df_test.drop(col_drop, axis=1, inplace=True)

In [14]:
len(df.columns)

366

In [15]:
len(df_test.columns)

365

In [16]:
df.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [17]:
#import xgBoost as xgb

In [18]:
#from sklearn.ensemble import XGBoostRegressor

In [19]:
# process columns, apply LabelEncoder to categorical features
from sklearn.preprocessing import LabelEncoder
for c in df.columns:
    if df[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(df[c].values) + list(df_test[c].values))
        df[c] = lbl.transform(df[c].values)
        df_test[c] = lbl.transform(df_test[c].values)

In [20]:
from sklearn.decomposition import PCA, FastICA

In [21]:
n_comp = 10
pca = PCA(n_components=n_comp, random_state = 42)
pca2_results_train = pca.fit_transform(df.drop(["ID", "y"], axis=1))
pca2_results_test = pca.transform(df_test.drop(["ID"], axis=1))
#pca2_results_train = pca.fit_transform(df.drop(["y"], axis=1))
#pca2_results_test = pca.transform(df_test)
# ICA
#ica = FastICA(n_components=n_comp, random_state=42)
#ica2_results_train = ica.fit_transform(df.drop(['ID','y'], axis=1))
#ica2_results_test = ica.transform(df_test.drop(["ID"], axis=1))

In [22]:
#x_train, x_test, y_train, y_test = train_test_split(pca2_results_train, df['y'], test_size=0.33, random_state=42)

In [23]:
#pca2_results_train.shape

In [24]:
# Append decomposition components to dataset
for i in range(1, n_comp+1):
    df['pca_'+str(i)] = pca2_results_train[:, i-1]
    df_test['pca_'+str(i)] = pca2_results_test[:, i-1]
#    df['ica_'+str(i)] = ica2_results_train[:, i-1]
#    df_test['ica_'+str(i)] = ica2_results_test[:, i-1]

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

In [26]:
reg = GradientBoostingRegressor(max_depth=5)

In [27]:
ans = reg.fit(df.drop(['ID', 'y'], axis=1), df['y'])
#ans = reg.fit(pca2_results_train, df['y'])
#ans = reg.fit(x_train, y_train)

In [28]:
out = reg.predict(df_test.drop(['ID'], axis=1))
#out = reg.predict(pca2_results_test)
#out = reg.predict(x_test)

In [29]:
#from sklearn.metrics import r2_score
#r2_score(y_test, out)

In [30]:
#out

In [31]:
df_out = pd.DataFrame()

In [32]:
df_out['ID'] = df_test['ID']

In [33]:
df_out['y'] = out

In [34]:
df_out.to_csv("out.csv", index=False)